## European Call Option with a fixed Knock-out

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime
import pandas_datareader as pdr
import datetime
import yfinance as yf
%matplotlib inline

In [2]:
start_date=datetime.datetime(2017,11,28)
end_date=datetime.datetime(2023,11,28)
tickers = 'TSLA'
prices = yf.download(tickers, start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [4]:
historical = prices['Adj Close'].dropna(how='all')

In [5]:
ret = historical.pct_change()[1:] #daily returns for the month

In [6]:
days = len(ret)
sample_mean_tsla = np.mean(ret)
sample_std_tsla = np.std(ret, ddof=1)

In [14]:
lastPrice= S0 = historical.iloc[-1]

## Here we keep the strike price of the European Option to 260, but set a knock-out at 280, this will limit the payoff, reduce the average of payoffs, hence reduce the price of the option

In [16]:
import numpy as np

# Function to price a European call option using Monte Carlo simulation
def monte_carlo_option_price(S, K, r, T, vol, num_simulations, num_results):
    dt = 1/252  # Time step, assuming 252 trading days in a year
    num_days = int(T * 252)  # Number of days until expiration

    # Array to store the final option prices from each set of simulations
    final_option_prices = np.zeros(num_results)

    for simulation in range(num_results):
        simulated_prices = np.zeros((num_simulations, num_days))

        for i in range(num_simulations):
            price_path = np.zeros(num_days)
            price_path[0] = S
            for day in range(1, num_days):
                drift = (r - 0.5 * vol ** 2) * dt
                diffusion = vol * np.sqrt(dt) * np.random.normal(0, 1)
                price_path[day] = price_path[day - 1] * np.exp(drift + diffusion)
                # Implementing the knock-out option feature
                if price_path[day] >= 280:
                    price_path[day:] = 0
                    break

            simulated_prices[i, :] = price_path

        # Calculate the payoff for each simulation at expiration
        option_payoffs = np.maximum(simulated_prices[:, -1] - K, 0)
        # Calculate the price of the option as the average of discounted payoffs
        option_price = np.exp(-r * T) * np.mean(option_payoffs)
        final_option_prices[simulation] = option_price

    # Calculate the average option price across all sets of simulations for higher precision
    average_option_price = np.mean(final_option_prices)
    return average_option_price, final_option_prices

# Option parameters
S0 = lastPrice  # Current stock price (assuming this is the value of S)
K = 260  # Strike price
r = 0.05  # Risk-free interest rate
T = 1  # Time to expiration (in years)
sample_std_tsla = sample_std_tsla  # An assumed value for Tesla's volatility
num_simulations = 1000  # Number of Monte Carlo simulations
num_results = 10  # Number of different simulation results to store

# European Knock-Out Call Option Price with Monte Carlo Simulation
average_option_price, all_option_prices = monte_carlo_option_price(S0, K, r, T, sample_std_tsla, num_simulations, num_results)

average_option_price, all_option_prices


(0.11790469559909474,
 array([0.16377305, 0.08588809, 0.11857446, 0.11018229, 0.16555139,
        0.08973333, 0.11221112, 0.09127628, 0.08969505, 0.15216189]))